# Tutorial: Genome-Scale Metabolic Modeling with COBRApy

## Overview

This tutorial demonstrates how to use genome-scale metabolic models (GEMs) to identify gene knockout/knockdown targets using COBRApy. We'll explore various constraint-based modeling techniques to find metabolic engineering targets.

## Objectives

1. Load and analyze genome-scale metabolic models
2. Perform single gene knockout analysis
3. Identify essential genes
4. Discover synthetic lethal gene pairs
5. Conduct flux variability analysis (FVA)
6. Find growth-coupled production targets
7. Visualize and interpret results

## What are Genome-Scale Metabolic Models?

GEMs are mathematical representations of an organism's complete metabolism:
- **Reactions**: All biochemical reactions
- **Metabolites**: All compounds involved
- **Genes**: Genes encoding enzymes
- **Stoichiometry**: Mass-balanced equations

### Constraint-Based Modeling

Uses mathematical optimization to predict cellular behavior:
- **Flux Balance Analysis (FBA)**: Find flux distributions that maximize/minimize an objective
- **Constraints**: Mass balance, reaction bounds, thermodynamics
- **No kinetics required**: Only stoichiometry and bounds

---

## Step 1: Environment Setup

In [ ]:
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import (
    flux_variability_analysis,
    single_gene_deletion,
    double_gene_deletion,
)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

print(f"COBRApy version: {cobra.__version__}")
print("Environment setup complete!")

## Step 2: Load a Metabolic Model

COBRApy can load models from:
- **BiGG Database**: Pre-built models (E. coli, human, yeast, etc.)
- **SBML files**: Standard format for metabolic models
- **JSON/MAT files**: COBRApy-specific formats

Let's start with the E. coli core model (small, fast):

In [ ]:
# Load E. coli core model (small model for demonstration)
model = cobra.io.load_model("textbook")

print(f"Model: {model.id}")
print(f"Reactions: {len(model.reactions)}")
print(f"Metabolites: {len(model.metabolites)}")
print(f"Genes: {len(model.genes)}")
print(f"\nObjective: {model.objective.expression}")

### Model Components

Let's explore the model structure:

In [ ]:
# Examine some reactions
print("=== Example Reactions ===")
for rxn in list(model.reactions)[:5]:
    print(f"\n{rxn.id}: {rxn.name}")
    print(f"  Equation: {rxn.reaction}")
    print(f"  Bounds: [{rxn.lower_bound}, {rxn.upper_bound}]")
    print(f"  Genes: {', '.join([g.id for g in rxn.genes])}")

In [ ]:
# Examine some metabolites
print("=== Example Metabolites ===")
for met in list(model.metabolites)[:5]:
    print(f"\n{met.id}: {met.name}")
    print(f"  Formula: {met.formula}")
    print(f"  Compartment: {met.compartment}")
    print(f"  Reactions: {len(met.reactions)}")

In [ ]:
# Examine some genes
print("=== Example Genes ===")
for gene in list(model.genes)[:5]:
    print(f"\n{gene.id}: {gene.name}")
    print(f"  Reactions: {', '.join([r.id for r in gene.reactions][:3])}...")
    print(f"  Total reactions: {len(gene.reactions)}")

## Step 3: Flux Balance Analysis (FBA)

FBA predicts cellular behavior by optimizing an objective function subject to constraints.

### Wildtype Growth Prediction

In [ ]:
# Optimize wildtype model
solution = model.optimize()

print(f"Optimal growth rate: {solution.objective_value:.4f} 1/h")
print(f"Status: {solution.status}")

# Show top fluxes
print("\n=== Top 10 Fluxes ===")
flux_series = solution.fluxes.abs().sort_values(ascending=False)
print(flux_series.head(10))

### Visualize Flux Distribution

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Flux magnitude distribution
ax = axes[0]
flux_values = solution.fluxes[solution.fluxes.abs() > 1e-6]
ax.hist(flux_values.abs(), bins=30, edgecolor='black', alpha=0.7)
ax.set_xlabel('Absolute Flux', fontsize=12)
ax.set_ylabel('Number of Reactions', fontsize=12)
ax.set_title('Distribution of Flux Magnitudes', fontsize=14, fontweight='bold')
ax.set_yscale('log')
ax.grid(alpha=0.3)

# Top fluxes bar plot
ax = axes[1]
top_fluxes = solution.fluxes.abs().sort_values(ascending=False).head(15)
bars = ax.barh(range(len(top_fluxes)), top_fluxes.values)
ax.set_yticks(range(len(top_fluxes)))
ax.set_yticklabels(top_fluxes.index, fontsize=9)
ax.set_xlabel('Absolute Flux', fontsize=12)
ax.set_title('Top 15 Reactions by Flux', fontsize=14, fontweight='bold')
ax.invert_yaxis()
ax.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

---

## Step 4: Single Gene Knockout Analysis

Systematically delete each gene and measure the effect on growth.

### Perform Single Gene Deletions

In [ ]:
print("Performing single gene knockout analysis...")
print(f"Testing {len(model.genes)} genes...\n")

# Run single gene deletion
deletion_results = single_gene_deletion(model)

# Process results
wildtype_growth = solution.objective_value
deletion_results['growth_fraction'] = deletion_results['growth'] / wildtype_growth
deletion_results['growth_reduction'] = 1 - deletion_results['growth_fraction']

print("Results:")
print(deletion_results.head(10))

### Categorize Genes

In [ ]:
# Define thresholds
essential_threshold = 0.1  # < 10% of wildtype growth

# Categorize
essential_genes = deletion_results[deletion_results['growth_fraction'] < essential_threshold]
growth_reducing = deletion_results[
    (deletion_results['growth_fraction'] >= essential_threshold) &
    (deletion_results['growth_reduction'] > 0.1)
]
minimal_effect = deletion_results[deletion_results['growth_reduction'] <= 0.1]

print(f"Gene Classification:")
print(f"  Essential: {len(essential_genes)} ({len(essential_genes)/len(deletion_results)*100:.1f}%)")
print(f"  Growth-reducing: {len(growth_reducing)} ({len(growth_reducing)/len(deletion_results)*100:.1f}%)")
print(f"  Minimal effect: {len(minimal_effect)} ({len(minimal_effect)/len(deletion_results)*100:.1f}%)")

print(f"\n=== Top 10 Essential Genes ===")
print(essential_genes.sort_values('growth_fraction').head(10))

### Visualize Knockout Effects

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax = axes[0]
ax.hist(deletion_results['growth_fraction'], bins=30, edgecolor='black', alpha=0.7)
ax.axvline(essential_threshold, color='red', linestyle='--',
          label=f'Essential threshold ({essential_threshold})')
ax.set_xlabel('Growth Fraction (vs wildtype)', fontsize=12)
ax.set_ylabel('Number of Genes', fontsize=12)
ax.set_title('Single Gene Knockout Effects', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

# Pie chart
ax = axes[1]
sizes = [len(essential_genes), len(growth_reducing), len(minimal_effect)]
labels = [f'Essential\n({len(essential_genes)})',
         f'Growth-reducing\n({len(growth_reducing)})',
         f'Minimal effect\n({len(minimal_effect)})']
colors = ['#ff6b6b', '#ffd93d', '#6bcf7f']

ax.pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%',
      startangle=90, textprops={'fontsize': 11})
ax.set_title('Gene Classification', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

---

## Step 5: Essential Gene Analysis

Essential genes are critical for survival. Let's analyze them in detail.

In [ ]:
# Add detailed information about essential genes
essential_info = []

for gene_id in essential_genes['ids']:
    gene = model.genes.get_by_id(gene_id)
    reactions = list(gene.reactions)
    
    essential_info.append({
        'gene_id': gene_id,
        'gene_name': gene.name,
        'n_reactions': len(reactions),
        'reactions': ', '.join([r.id for r in reactions[:3]]),
        'growth_fraction': essential_genes[essential_genes['ids'] == gene_id]['growth_fraction'].iloc[0]
    })

essential_df = pd.DataFrame(essential_info)
essential_df = essential_df.sort_values('growth_fraction')

print("=== Essential Genes Analysis ===")
print(essential_df.to_string(index=False))

In [ ]:
# Visualize essential genes
fig, ax = plt.subplots(figsize=(10, 6))

if len(essential_df) > 0:
    top_essential = essential_df.head(20)
    bars = ax.barh(range(len(top_essential)), top_essential['growth_fraction'])
    ax.set_yticks(range(len(top_essential)))
    ax.set_yticklabels(top_essential['gene_id'], fontsize=9)
    ax.set_xlabel('Growth Fraction', fontsize=12)
    ax.set_title('Top 20 Most Critical Essential Genes', fontsize=14, fontweight='bold')
    ax.invert_yaxis()
    ax.grid(axis='x', alpha=0.3)
    
plt.tight_layout()
plt.show()

---

## Step 6: Flux Variability Analysis (FVA)

FVA identifies the range of possible flux values for each reaction while maintaining optimal (or near-optimal) growth.

In [ ]:
print("Performing Flux Variability Analysis...")
print("This may take a minute...\n")

# Perform FVA at 95% of optimal growth
fva_results = flux_variability_analysis(model, fraction_of_optimum=0.95)

# Calculate flux spans
fva_results['flux_span'] = fva_results['maximum'] - fva_results['minimum']
fva_results['flux_mid'] = (fva_results['maximum'] + fva_results['minimum']) / 2

print("FVA Results Summary:")
print(fva_results.head(10))

In [ ]:
# Identify reaction categories
zero_flux = fva_results[(fva_results['minimum'] == 0) & (fva_results['maximum'] == 0)]
essential_flux = fva_results[(fva_results['minimum'] > 0) | (fva_results['maximum'] < 0)]
flexible = fva_results[fva_results['flux_span'] > 1.0]

print(f"\nReaction Categories:")
print(f"  Zero flux (unused): {len(zero_flux)}")
print(f"  Essential flux: {len(essential_flux)}")
print(f"  Highly flexible (span > 1): {len(flexible)}")

print(f"\n=== Top 10 Most Flexible Reactions ===")
print(fva_results.nlargest(10, 'flux_span')[['minimum', 'maximum', 'flux_span']])

In [ ]:
# Visualize FVA results
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Flux span distribution
ax = axes[0]
ax.hist(fva_results['flux_span'], bins=50, edgecolor='black', alpha=0.7)
ax.set_xlabel('Flux Span (max - min)', fontsize=12)
ax.set_ylabel('Number of Reactions', fontsize=12)
ax.set_title('FVA: Flux Variability Distribution', fontsize=14, fontweight='bold')
ax.set_yscale('log')
ax.grid(alpha=0.3)

# Min vs Max
ax = axes[1]
ax.scatter(fva_results['minimum'], fva_results['maximum'], alpha=0.5, s=20)
ax.plot([-50, 50], [-50, 50], 'r--', alpha=0.5, label='No flexibility')
ax.set_xlabel('Minimum Flux', fontsize=12)
ax.set_ylabel('Maximum Flux', fontsize=12)
ax.set_title('FVA: Min vs Max Flux', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

---

## Step 7: Gene Knockout for Metabolite Production

Let's identify knockouts that could enhance production of a target metabolite.

Example: Acetate production

In [ ]:
# Target: Acetate (ac_c)
target_metabolite = 'ac_c'

# Check if exchange reaction exists
exchange_rxn_id = 'EX_ac_e'

if exchange_rxn_id in model.reactions:
    print(f"Exchange reaction {exchange_rxn_id} already exists")
    exchange_rxn = model.reactions.get_by_id(exchange_rxn_id)
else:
    print(f"Creating exchange reaction for {target_metabolite}")
    # Would need to create exchange reaction

print(f"\nExchange reaction: {exchange_rxn.reaction}")
print(f"Bounds: [{exchange_rxn.lower_bound}, {exchange_rxn.upper_bound}]")

In [ ]:
# Test knockouts for production
production_targets = []

# Test non-essential genes that reduce growth
test_genes = growth_reducing['ids'].head(20)  # Top 20 candidates

for gene_id in test_genes:
    with model:
        gene = model.genes.get_by_id(gene_id)
        gene.knock_out()
        
        solution = model.optimize()
        
        if solution.status == 'optimal':
            growth = solution.objective_value
            production = solution.fluxes[exchange_rxn_id] if exchange_rxn_id in solution.fluxes else 0
            
            production_targets.append({
                'gene_id': gene_id,
                'growth': growth,
                'production': abs(production),
                'ratio': abs(production) / growth if growth > 0 else 0
            })

prod_df = pd.DataFrame(production_targets)
prod_df = prod_df.sort_values('ratio', ascending=False)

print("=== Top Production-Enhancing Knockouts ===")
print(prod_df.head(10).to_string(index=False))

---

## Step 8: Using the Complete Analysis Script

Now let's use the comprehensive metabolic_target_finder.py script.

In [ ]:
# Import the analysis class
import sys
sys.path.append('../scripts')

from metabolic_target_finder import MetabolicTargetFinder, Args

# Configure analysis
args = Args(
    model_id="textbook",  # E. coli core model
    output_dir="../metabolic_results",
    ko_methods=["single", "essential", "fva"],
    growth_threshold=0.1,
    top_n_targets=20,
    visualization=True
)

# Run analysis
finder = MetabolicTargetFinder(args)
finder.load_model()

print("\n" + "="*80)
print("Running comprehensive metabolic analysis...")
print("="*80)

In [ ]:
# Single gene knockout
single_ko_results = finder.method_single_gene_knockout()

In [ ]:
# Essential genes
essential_results = finder.method_essential_genes()

In [ ]:
# FVA
fva_results = finder.method_flux_variability_analysis()

In [ ]:
# Identify best targets
best_targets = finder.identify_best_targets(criteria="growth_reduction")

In [ ]:
# Save results
finder.save_results(args.output_dir)
finder.visualize_results(args.output_dir)

print(f"\nResults saved to: {args.output_dir}/")

---

## Step 9: Advanced Analysis - Larger Models

For real applications, use larger genome-scale models:

### Available Models in BiGG Database:

**E. coli:**
- `iML1515`: Latest E. coli model (2,712 reactions, 1,877 genes)
- `iJO1366`: Previous E. coli model (2,583 reactions, 1,366 genes)

**Human:**
- `Recon3D`: Human metabolism (13,543 reactions, 3,288 genes)
- `Recon2`: Previous human model

**Yeast:**
- `iMM904`: S. cerevisiae (1,577 reactions, 904 genes)

**Others:**
- Many bacterial, fungal, and plant models

### Loading Larger Models:

```python
# Load E. coli iML1515
model = cobra.io.load_model("iML1515")

# Or from file
model = cobra.io.read_sbml_model("path/to/model.xml")
```

**Note**: Larger models take longer to analyze:
- Single KO: ~5-30 minutes
- Double KO: Hours to days (combinatorial)
- FVA: ~10-60 minutes

---

## Conclusion

### Summary

In this tutorial, we demonstrated:

1. **Loading metabolic models** from BiGG database or files
2. **Flux Balance Analysis** to predict growth and fluxes
3. **Single gene knockout** to identify essential and target genes
4. **Essential gene analysis** for drug target discovery
5. **Flux Variability Analysis** to explore metabolic flexibility
6. **Production enhancement** through gene knockouts

### Applications

**Metabolic Engineering:**
- Biofuel production
- Pharmaceutical synthesis
- Chemical manufacturing

**Drug Discovery:**
- Antibiotic target identification
- Cancer metabolism targets
- Synthetic lethality

**Systems Biology:**
- Understanding metabolism
- Predicting phenotypes
- Network analysis

### Next Steps

1. **Try larger models** (iML1515, Recon3D)
2. **Optimize for production** of specific compounds
3. **Explore synthetic lethality** with double knockouts
4. **Integrate with omics data** (transcriptomics, proteomics)
5. **Design metabolic engineering strategies**

### Resources

- **COBRApy documentation**: https://cobrapy.readthedocs.io/
- **BiGG Models**: http://bigg.ucsd.edu/
- **SBML**: http://sbml.org/
- **Systems Biology**: https://www.systemsbiology.org/

---

**For questions or issues, please refer to the Virtual Lab documentation or create an issue on GitHub.**